In [ ]:
using Revise 

In [ ]:
using Cairn 
using LinearAlgebra, Random, Statistics, StatsBase # Do need StatsBase this time
using PotentialLearning 
using Molly, AtomsCalculators
using AtomisticQoIs 
using SpecialPolynomials


In [ ]:
include("/Users/swyant/cesmix/dev/Cairn.jl/src/makie/makie.jl")

In [ ]:
ref = MullerBrownRot() 

temp = 100.0u"K"
x0 = [-1.5,1.0]
sys0 = System( 
        ref, 
        x0,
        loggers=(coords=CoordinateLogger(1000;dims=2),),
)

limits = [[-4.4,1.5],[-2,2]]
coord_grid = coord_grid_2d(limits, 0.05)
ctr_lvls = -150:20:400

In [ ]:
coords_eval = potential_grid_2d(ref,limits,0.05,cutoff=1000)
sys_eval = Ensemble(ref,coords_eval)

ξ = [ustrip.(Vector(coords)) for coords in coords_eval]
GQint = GaussQuadrature(ξ, ones(length(ξ))./length(ξ))

In [ ]:
f0, ax0 = plot_contours_2d(ref, coord_grid; fill=true, lvls=ctr_lvls)
coordmat = reduce(hcat, [get_values(crd) for crd in coords_eval])'
scatter!(ax0, coordmat[:,1], coordmat[:,2], color=:red, markersize=5, label="test points")
axislegend(ax0)
f0

In [ ]:
sim_langevin = OverdampedLangevin(
            dt=0.001u"ps",
            temperature=1200.0u"K",
            friction=4.0u"ps^-1")

sys = deepcopy(sys0)
simulate!(sys, sim_langevin, 1_000_000)

In [ ]:
f = plot_md_trajectory(sys, coord_grid, fill=false, lvls=ctr_lvls, showpath=false)

In [ ]:
nens = 100
id = StatsBase.sample(1:length(sys.loggers.coords.history), nens, replace=false)
coords0 = [sys.loggers.coords.history[i][1] for i in id]

ens0 = Ensemble(ref, coords0,
    loggers=(
        coords=CoordinateLogger(10; dims=2),
    ),
)

In [ ]:
rbf = RBF(Euclidean(2), β=1.0, ℓ=1.0)
sim_svgd = StochasticSVGD(
            dt=0.001u"ps",
            kernel=rbf,
            kernel_bandwidth=median_kernel_bandwidth,
            temperature=20.0u"K",
            friction=1.0u"ps^-1")

simulate!(ens0, sim_svgd, 10_000)


In [ ]:
f = plot_md_trajectory(ens0, coord_grid, fill=false, lvls=ctr_lvls, showpath=false)

In [ ]:
coords_train = coords0
sys_train = Ensemble(ref, coords_train,
    data=Dict(
        "energy_descriptors" => [],
        "force_descriptors" => [],
    )
)

In [ ]:
# define model
basisfam = Jacobi{0.5,0.5} # ChebyshevU
order = 8 # [5,10,15,20,25,30]
pce = PolynomialChaos(order, 2, basisfam, xscl=limits)

pce0 = deepcopy(pce)

nens = 10
ens0 = Ensemble(pce0, coords_train[1:nens],
    loggers=(
        coords=CoordinateLogger(10; dims=2),
        steps=StepComponentLogger(10; dims=2),
    ),
    data=Dict(
        "energy_descriptors" => Float64[],
        "force_descriptors" => Vector[],
        "kernel" => 1.0,
        "ksd" => 1.0,
    )
)

train!(ens0, sys_train, ref) # trains 
pce0.params = ens0[1].general_inters[1].params

In [ ]:
ctr_lvls0 = -300:20:100
f0, ax0 = plot_contours_2d(pce0, coord_grid; fill=true, lvls=ctr_lvls0)
coordmat = reduce(hcat, [get_values(crd) for crd in coords_train])'
scatter!(ax0, coordmat[:,1], coordmat[:,2], color=:red, label="training points")
axislegend(ax0)
f0


Already this example is kind of bricked because plot_contours_2d calls potential energy, which calls compute_local_descriptors, for which there is no longer the method handle of (coords, pce,) but instead (sys, pce)

In [ ]:
rbf = RBF(Euclidean(2), β=0.2)


# define simulator
sim_svgd = StochasticSVGD(
            dt=0.001u"ps",
            kernel=rbf,
            kernel_bandwidth=median_kernel_bandwidth,
            sys_fix=sys_train,
            temperature=temp,
            friction=1.0u"ps^-1")

# simulate
ens = deepcopy(ens0)
simulate!(ens, sim_svgd, 1_000)


# define triggers
trig1 = TimeInterval(interval=100)
trig2 = MaxVol(thresh=1.22)

In [ ]:
#this is totally undefined
al = ActiveLearnRoutine(
    ref=ref,
    mlip=pce0,
    trainset=sys_train,
    triggers=(trig1, trig2),
    dataselector=RandomSelector,
    trainobj=LinearLeastSquares,
)